## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-04-05-35-28 +0000,nyt,Lisbon Funicular Crash: What to Know About Cau...,https://www.nytimes.com/2025/09/04/world/europ...
1,2025-09-04-05-31-18 +0000,nypost,Woman with ‘fascination for firearms’ arrested...,https://nypost.com/2025/09/04/us-news/las-vega...
2,2025-09-04-05-21-50 +0000,bbc,Price cut for some Firmus gas customers,https://www.bbc.com/news/articles/c7v15e89p81o...
3,2025-09-04-05-08-41 +0000,bbc,Council scheme to fight rising costs ahead of ...,https://www.bbc.com/news/articles/c89dwvq20nqo...
4,2025-09-04-05-00-00 +0000,wapo,Inside the Trump team’s conflicting efforts to...,https://www.washingtonpost.com/world/2025/09/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,47
67,china,22
187,putin,16
828,new,16
69,military,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
216,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...,127
27,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...,112
157,2025-09-03-19-50-17 +0000,nypost,Trump rips reporter for saying he’s taken ‘no ...,https://nypost.com/2025/09/03/us-news/trump-ri...,111
326,2025-09-03-08-26-24 +0000,nyt,"China’s Military Parade, in Photos: Xi Unveils...",https://www.nytimes.com/2025/09/03/world/asia/...,108
24,2025-09-04-03-23-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
216,127,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...
27,65,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...
255,63,2025-09-03-14-32-35 +0000,nypost,Kim Jong Un is seen with his young daughter in...,https://nypost.com/2025/09/03/world-news/kim-j...
51,59,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
102,53,2025-09-03-22-33-42 +0000,nypost,JD and Usha Vance pay their respects to victim...,https://nypost.com/2025/09/03/us-news/jd-and-u...
221,49,2025-09-03-16-30-19 +0000,nypost,"Top chefs Daniel Boulud, Jesus Duron cook up n...",https://nypost.com/2025/09/03/business/chefs-d...
72,47,2025-09-04-00-00-00 +0000,wsj,Hundreds of current and former Health and Huma...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
206,44,2025-09-03-17-16-35 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...
171,40,2025-09-03-19-22-00 +0000,wsj,Newsmax Files Antitrust Lawsuit Against Fox News,https://www.wsj.com/business/media/newsmax-fil...
107,38,2025-09-03-22-14-19 +0000,latimes,Alphabet's stock climbs after judge rules Goog...,https://www.latimes.com/business/story/2025-09...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
